In [1]:
from win32com.client import Dispatch
import time
import numpy as np
import pulsar_dimmer

In [2]:
maxim = Dispatch("MaxIm.Application")
maxim.LockApp = True

In [3]:
camera = Dispatch("MaxIm.CCDCamera")
camera.DisableAutoShutdown = True
camera.LinkEnabled = True
print('Camera connected: %s' % camera.CameraName); print()

readouts = camera.ReadoutModes
for i in range(len(readouts)):
    print('Mode %d: %s' % (i, readouts[i]))
print()
filter_names = camera.FilterNames
for i in range(len(filter_names)):
    print('Filter %d: %s' % (i, filter_names[i]))

Camera connected: DL Imaging

Mode 0: HDR
Mode 1: High Gain
Mode 2: Low Gain
Mode 3: High Gain StackPro

Filter 0: L
Filter 1: 6
Filter 2: V
Filter 3: B
Filter 4: H
Filter 5: W
Filter 6: O
Filter 7: 1
Filter 8: I
Filter 9: X
Filter 10: G
Filter 11: R


In [4]:
mount = Dispatch('SiTech.Telescope')

In [5]:
print('Homing')
mount.FindHome
mount.Tracking = False
print('Home found')

Homing
Home found


In [6]:
# mount.Park

print('Slewing')
mount.SlewToAltAz(86.96717, 30.09397)
print('Slew complete')
mount.Tracking = False

Slewing
Slew complete


In [7]:
print('Setting lights')
pulsar_dimmer.dimmer(1)
print('Lights set')

Setting lights
Lights set


In [8]:
save_loc = 'D:/Calibrations/Testing_4-28-2023/'
readoutLoop = [3]
binLoop = [1]
exposureLoop = [32]
filLoop = [0]
light = [0, 1]; im_type = {0:'dark', 1:'flat'}
repeat = 25 # number of images per option
estimatedTime = len(readoutLoop)*len(binLoop)*len(filLoop)*len(light)*repeat*sum(exposureLoop)/(60*60)
print('Estimated time to complete: %.2f hours' % estimatedTime)

Estimated time to complete: 0.44 hours


In [9]:
for fil in filLoop:
    for shutter in light:
        for binning in binLoop:
            camera.BinX = binning
            for readout in readoutLoop:
                camera.ReadoutMode = readout
                for exp in exposureLoop:
                    for i in range(repeat):
                        while camera.Temperature > -19.9:
                            print('Waiting for temperature cooldown...')
                            time.sleep(5)
                        camera.Expose(exp, shutter, fil)
                        savestring = save_loc + (im_type[shutter]+'_'+
                                        camera.ReadoutModes[readout].replace(' ', '')+'_'+
                                        str(binning)+'x'+str(binning)+'_'+
                                        camera.Filternames[fil]+'_'+
                                        str(exp).replace('.', '')+'s__'+
                                        str(i)+'.fts')
                        while not camera.ImageReady:
                            time.sleep(0.1)
                        print('Saving: %s' % savestring)
                        camera.SaveImage(savestring)

print('Turning off lights')
pulsar_dimmer.dimmer(0)
print('Lights off')

mount.Park

Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__0.fts
Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__1.fts
Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__2.fts
Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__3.fts
Waiting for temperature cooldown...
Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__4.fts
Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__5.fts
Waiting for temperature cooldown...
Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__6.fts
Waiting for temperature cooldown...
Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__7.fts
Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__8.fts
Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__9.fts
Saving: D:/Calibrations/Testing_4-28-2023/dark_HighGainStackPro_1x1_L_32s__10.fts


0